In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.drbart_evaluation import *

In [2]:
log_name = 'test'
model_name = 'pcr_2'
n_processes = 20

with open('../transformed_event_logs/PCR_start_end_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)


known_activities = ['Callback timeout', 'Export result', 'Export to EMS', 'Match patient data', 'Receive sample state', 'Send notification', 'Wait for plate validation', 'timeout']

likelihoods_A = None
likelihoods_A_S = None
likelihoods_A_S_AC = None

/tmp/ipykernel_34827/1108454906.py:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
N = 1000
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [4]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_PCR_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                      | 0/1235 [00:00<?, ?it/s]

  0%|                            | 1/1235 [00:29<9:56:46, 29.02s/it]

  0%|                            | 2/1235 [00:29<4:17:18, 12.52s/it]

  0%|                            | 3/1235 [00:30<2:28:14,  7.22s/it]

  0%|                            | 4/1235 [00:31<1:36:29,  4.70s/it]

...

 94%|████████████████████████▍ | 1161/1235 [00:07<00:00, 164.49it/s]

 95%|████████████████████████▊ | 1178/1235 [00:07<00:00, 156.09it/s]

 97%|█████████████████████████▏| 1194/1235 [00:07<00:00, 128.18it/s]

 99%|█████████████████████████▊| 1224/1235 [00:07<00:00, 170.08it/s]

100%|██████████████████████████| 1235/1235 [00:08<00:00, 153.73it/s]

In [5]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('0.1692347031725720576684346534')

In [6]:
np.mean(get_pscores(likelihoods_A))

np.float64(15212.073667727887)

In [7]:
drbart_model_A_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day/',
                     strict_parser=True)
evaluator_A_S = conduct_evaluation.ConductEvaluation(drbart_model_A_S, SampleOutcomes_DRBART_PCR_A_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S = evaluator_A_S.sample_cases(False, True)

  0%|                                      | 0/1235 [00:00<?, ?it/s]

  0%|                           | 1/1235 [00:36<12:24:37, 36.21s/it]

  0%|                            | 3/1235 [00:37<3:20:49,  9.78s/it]

  0%|                            | 4/1235 [00:38<2:19:02,  6.78s/it]

  0%|                            | 5/1235 [00:39<1:39:41,  4.86s/it]

...

 94%|████████████████████████▍ | 1162/1235 [00:07<00:00, 172.34it/s]

 96%|████████████████████████▊ | 1181/1235 [00:07<00:00, 174.01it/s]

 97%|█████████████████████████▏| 1199/1235 [00:07<00:00, 169.45it/s]

 99%|█████████████████████████▌| 1217/1235 [00:07<00:00, 166.61it/s]

100%|██████████████████████████| 1235/1235 [00:08<00:00, 153.15it/s]

In [8]:
np.mean([v.ln() for v in likelihoods_A_S[0].values()])

Decimal('0.6278662372456164028905250418')

In [9]:
np.mean(get_pscores(likelihoods_A_S))

np.float64(11993.507890950912)

In [10]:
drbart_model_A_S_D = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day_day-of-week/',
                     strict_parser=True)
evaluator_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_A_S_D, SampleOutcomes_DRBART_PCR_A_S_D,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'day_of_week' : 'day_of_week',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S_D = evaluator_A_S_D.sample_cases(False, True)

  0%|                                      | 0/1235 [00:00<?, ?it/s]

  0%|                           | 1/1235 [00:35<12:02:48, 35.14s/it]

  0%|                            | 2/1235 [00:37<5:23:56, 15.76s/it]

  0%|                            | 3/1235 [00:40<3:25:06,  9.99s/it]

  0%|                            | 5/1235 [00:41<1:37:15,  4.74s/it]

...

 96%|████████████████████████▉ | 1182/1235 [00:07<00:00, 173.62it/s]

 97%|█████████████████████████▎| 1200/1235 [00:07<00:00, 164.39it/s]

 99%|█████████████████████████▌| 1217/1235 [00:07<00:00, 165.91it/s]

100%|█████████████████████████▉| 1234/1235 [00:07<00:00, 166.11it/s]

100%|██████████████████████████| 1235/1235 [00:07<00:00, 156.11it/s]

In [11]:
np.mean([v.ln() for v in likelihoods_A_S_D[0].values()])

Decimal('-0.3612432211840162706955061669')

In [12]:
np.mean(get_pscores(likelihoods_A_S_D))

np.float64(9786.73284525094)

In [13]:
drbart_model_A_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day_activity-count/',
                     strict_parser=False)
evaluator_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_S_AC, SampleOutcomes_DRBART_PCR_A_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'known_activities' : known_activities,
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S_AC = evaluator_A_S_AC.sample_cases(False, True)

  0%|                                      | 0/1235 [00:00<?, ?it/s]

  0%|                            | 1/1235 [00:25<8:37:29, 25.16s/it]

  0%|                            | 2/1235 [00:26<3:50:08, 11.20s/it]

  0%|                            | 4/1235 [00:27<1:30:48,  4.43s/it]

  0%|                            | 5/1235 [00:30<1:25:13,  4.16s/it]

...

 95%|████████████████████████▊ | 1178/1235 [00:07<00:00, 172.29it/s]

 97%|█████████████████████████▏| 1196/1235 [00:07<00:00, 168.32it/s]

 98%|█████████████████████████▌| 1215/1235 [00:08<00:00, 170.17it/s]

100%|█████████████████████████▉| 1234/1235 [00:08<00:00, 171.84it/s]

100%|██████████████████████████| 1235/1235 [00:08<00:00, 152.14it/s]

In [14]:
cases = test_event_log['case:concept:name'].unique()
case_data = dict([(case_name, evaluator_A_S_AC.get_case_data(case_name)) for case_name in cases])

In [15]:
np.mean([v.ln() for v in likelihoods_A_S_AC[0].values()])

Decimal('0.5134001219009783543864833947')

In [16]:
np.mean(get_pscores(likelihoods_A_S_AC))

np.float64(13683.480426913726)

In [17]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_S' : likelihoods_A_S,
    'drbart_model_A_S_D' : likelihoods_A_S_D,
    'drbart_model_A_S_AC' : likelihoods_A_S_AC
}
with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)